# Sessions & Memory Demo

**Demonstrating how to use saved sessions and memory for personalized experiences.**

This notebook shows practical examples of:
1. Creating sessions with user preferences
2. Resuming previous sessions
3. Using memory for personalization
4. Querying session history
5. Building user profiles over time

## Setup

In [10]:
import os
import uuid
import json
import sqlite3
from dotenv import load_dotenv

from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner

from services.session_service import create_session_service
from services.memory_service import create_memory_service
from tools.google_books import google_books_tool
from agents.orchestrator import create_workflow
from models.preferences import TravelPreferences

load_dotenv()

print("✅ Imports complete")
print(f"   Google API Key: {'OK' if os.getenv('GOOGLE_API_KEY') else 'MISSING'}")

✅ Imports complete
   Google API Key: OK


## Scenario 1: First-Time User with Preferences

Create a new user session with travel preferences that will persist.

In [11]:
# Initialize services with database
session_service = create_session_service(use_database=True)
memory_service = create_memory_service()

# Configure model
retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model = Gemini(
    model="gemini-2.0-flash-lite",
    api_key=os.getenv('GOOGLE_API_KEY'),
    retry_options=retry_config
)

print("✅ Services initialized")

Using DatabaseSessionService with: sqlite:///storyland_sessions.db
Using InMemoryMemoryService (keyword search)
✅ Services initialized


In [12]:
# Create first session with user preferences
session_1_id = str(uuid.uuid4())
user_id = "alice"

# Define user preferences
user_preferences = {
    "prefers_museums": True,
    "travels_with_kids": False,
    "budget": "moderate",
    "favorite_genres": ["historical fiction", "classics"],
    "dietary_restrictions": ["vegetarian"],
    "preferred_pace": "relaxed"
}

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_1_id,
    state={
        "book_title": "Pride and Prejudice",
        "author": "Jane Austen",
        "user:preferences": user_preferences  # User-scoped!
    }
)

print(f"✅ Session 1 created for user '{user_id}'")
print(f"   Session ID: {session_1_id[:8]}...")
print(f"   Book: Pride and Prejudice")
print(f"   Preferences saved:")
print(f"     - Museums: {user_preferences['prefers_museums']}")
print(f"     - Budget: {user_preferences['budget']}")
print(f"     - Pace: {user_preferences['preferred_pace']}")

✅ Session 1 created for user 'alice'
   Session ID: cd4b7f04...
   Book: Pride and Prejudice
   Preferences saved:
     - Museums: True
     - Budget: moderate
     - Pace: relaxed


### Run the workflow for Session 1

This would normally create a full itinerary. For demo purposes, we'll simulate it.

In [13]:
# Simulate workflow completion by adding results to state
# In a real scenario, you'd run the full workflow

session_1 = await session_service.get_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_1_id
)

# Simulate adding itinerary results
session_1.state["final_itinerary"] = {
    "cities": [
        {
            "name": "Bath",
            "country": "England",
            "days_suggested": 2,
            "overview": "Explore the Regency-era city that inspired Jane Austen",
            "stops": [
                {
                    "name": "Jane Austen Centre",
                    "type": "museum",
                    "reason": "Learn about Jane Austen's life in Bath",
                    "time_of_day": "morning"
                },
                {
                    "name": "Roman Baths",
                    "type": "landmark",
                    "reason": "Historic site mentioned in Austen's works",
                    "time_of_day": "afternoon"
                }
            ]
        }
    ],
    "summary_text": "A relaxed journey through Jane Austen's England with museum visits."
}

# Save to memory
await memory_service.add_session_to_memory(session_1)

print("✅ Session 1 completed and saved to memory")
print("   User preferences are now persistent!")

✅ Session 1 completed and saved to memory
   User preferences are now persistent!


## Scenario 2: Returning User - Preferences Auto-Loaded

When the same user creates a new session, their preferences automatically carry over.

In [14]:
# Create second session for same user
session_2_id = str(uuid.uuid4())

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,  # Same user!
    session_id=session_2_id,
    state={
        "book_title": "Gone with the Wind",
        "author": "Margaret Mitchell"
        # Notice: No preferences specified!
    }
)

# Get the session and check preferences
session_2 = await session_service.get_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_2_id
)

# Check if preferences are there
loaded_preferences = session_2.state.get("user:preferences")

print(f"✅ Session 2 created for user '{user_id}'")
print(f"   Session ID: {session_2_id[:8]}...")
print(f"   Book: Gone with the Wind")
print(f"\n🎯 Preferences automatically loaded from previous session:")

if loaded_preferences:
    print(json.dumps(loaded_preferences, indent=2))
else:
    print("   ⚠️  No preferences found (check database persistence)")

✅ Session 2 created for user 'alice'
   Session ID: fdff9172...
   Book: Gone with the Wind

🎯 Preferences automatically loaded from previous session:
{
  "prefers_museums": true,
  "travels_with_kids": false,
  "budget": "moderate",
  "favorite_genres": [
    "historical fiction",
    "classics"
  ],
  "dietary_restrictions": [
    "vegetarian"
  ],
  "preferred_pace": "relaxed"
}


## Scenario 3: Query Memory for Personalization

Use memory service to find relevant past interactions.

In [15]:
# Search memory for past travel preferences
memory_results = await memory_service.search_memory(
    app_name="storyland",
    user_id=user_id,
    query="travel preferences museums budget"
)

print(f"🔍 Memory search results:")
print(f"   Found {len(memory_results.memories)} relevant memories")

if memory_results.memories:
    print(f"\n   Latest memory:")
    latest = memory_results.memories[0]
    print(f"     - Author: {latest.author}")
    print(f"     - Timestamp: {latest.timestamp}")
    print(f"     - Content preview: {str(latest.content)[:200]}...")
else:
    print("   No memories found yet")

🔍 Memory search results:
   Found 0 relevant memories
   No memories found yet


## Scenario 4: Resume a Previous Session

Continue working on an existing itinerary.

In [16]:
# Load session 1 again
resumed_session = await session_service.get_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_1_id  # Original session ID
)

print(f"📂 Resumed session: {session_1_id[:8]}...")
print(f"\n   Book: {resumed_session.state.get('book_title')}")
print(f"   Author: {resumed_session.state.get('author')}")

# Check if itinerary exists
itinerary = resumed_session.state.get("final_itinerary")
if itinerary:
    print(f"\n   ✅ Previous itinerary found:")
    print(f"      Cities: {len(itinerary.get('cities', []))}")
    print(f"      Summary: {itinerary.get('summary_text')[:100]}...")
    
    # You could now modify this itinerary
    # For example, add a new city or refine existing stops
else:
    print(f"\n   ⚠️  No itinerary found in this session")

print(f"\n   Number of events in conversation: {len(resumed_session.events)}")

📂 Resumed session: cd4b7f04...

   Book: Pride and Prejudice
   Author: Jane Austen

   ⚠️  No itinerary found in this session

   Number of events in conversation: 0


## Scenario 5: Query Database Directly

Use SQL to analyze user behavior and history.

In [17]:
def get_user_history(user_id: str):
    """Get all books a user has explored."""
    conn = sqlite3.connect('storyland_sessions.db')
    cursor = conn.cursor()
    
    # Query sessions for this user
    cursor.execute("""
        SELECT 
            id,
            json_extract(state, '$.book_title') as book_title,
            json_extract(state, '$.author') as author,
            create_time
        FROM sessions
        WHERE user_id = ?
        ORDER BY create_time DESC
    """, (user_id,))
    
    results = cursor.fetchall()
    conn.close()
    return results

# Get Alice's history
history = get_user_history(user_id)

print(f"📚 User '{user_id}' has explored {len(history)} books:")
print()

for i, (sess_id, book, author, created) in enumerate(history, 1):
    print(f"   {i}. {book or 'Unknown'} by {author or 'Unknown'}")
    print(f"      Session: {sess_id[:8]}...")
    print(f"      Created: {created}")
    print()

📚 User 'alice' has explored 5 books:

   1. Pride and Prejudice by Jane Austen
      Session: cd4b7f04...
      Created: 2025-11-21 03:38:36

   2. Gone with the Wind by Margaret Mitchell
      Session: fdff9172...
      Created: 2025-11-21 03:38:36

   3. Pride and Prejudice by Jane Austen
      Session: 06544343...
      Created: 2025-11-21 03:34:42

   4. Gone with the Wind by Margaret Mitchell
      Session: c2375296...
      Created: 2025-11-21 03:34:42

   5. The Nightingale by Unknown
      Session: 13080961...
      Created: 2025-11-21 03:26:04



## Scenario 6: Build User Profile Over Time

Analyze preferences and behavior to create a rich user profile.

In [23]:
def get_user_profile(user_id: str):
    """Build a user profile from database."""
    conn = sqlite3.connect('storyland_sessions.db')
    cursor = conn.cursor()
    
    # Get all sessions
    cursor.execute("""
        SELECT 
            COUNT(*) as total_sessions,
            MIN(create_time) as first_session,
            MAX(create_time) as last_session
        FROM sessions
        WHERE user_id = ?
    """, (user_id,))
    
    stats = cursor.fetchone()
    
    # Get preferences from most recent session
    cursor.execute("""
        SELECT json_extract(state, '$."user:preferences"') as preferences
        FROM sessions
        WHERE user_id = ?
        ORDER BY create_time DESC
        LIMIT 1
    """, (user_id,))
    
    prefs_row = cursor.fetchone()
    preferences = json.loads(prefs_row[0]) if prefs_row and prefs_row[0] else {}
    
    # Get all books explored
    cursor.execute("""
        SELECT json_extract(state, '$.book_title') as book
        FROM sessions
        WHERE user_id = ?
        ORDER BY create_time
    """, (user_id,))
    
    books = [row[0] for row in cursor.fetchall() if row[0]]
    
    conn.close()
    
    return {
        "user_id": user_id,
        "total_sessions": stats[0],
        "first_visit": stats[1],
        "last_visit": stats[2],
        "preferences": preferences,
        "books_explored": books
    }

# Build profile
profile = get_user_profile(user_id)

print(f"👤 USER PROFILE: {profile['user_id']}")
print("=" * 60)
print(f"\n📊 Activity:")
print(f"   Total sessions: {profile['total_sessions']}")
print(f"   First visit: {profile['first_visit']}")
print(f"   Last visit: {profile['last_visit']}")

print(f"\n📚 Books Explored ({len(profile['books_explored'])}::")
for book in profile['books_explored']:
    print(f"   - {book}")

print(f"\n⚙️  Preferences:")
if profile['preferences']:
    for key, value in profile['preferences'].items():
        print(f"   - {key}: {value}")
else:
    print("   No preferences set")

👤 USER PROFILE: alice

📊 Activity:
   Total sessions: 6
   First visit: 2025-11-21 03:26:04
   Last visit: 2025-11-21 03:38:36

📚 Books Explored (6::
   - The Nightingale
   - Pride and Prejudice
   - Gone with the Wind
   - Pride and Prejudice
   - Gone with the Wind
   - Harry Potter and the Philosopher's Stone

⚙️  Preferences:
   No preferences set


## Scenario 7: Update User Preferences

User preferences can be updated in new sessions.

In [24]:
# User decides they now travel with kids
session_3_id = str(uuid.uuid4())

# Update preferences
updated_preferences = user_preferences.copy()
updated_preferences["travels_with_kids"] = True
updated_preferences["preferred_pace"] = "moderate"  # Changed from relaxed

await session_service.create_session(
    app_name="storyland",
    user_id=user_id,
    session_id=session_3_id,
    state={
        "book_title": "Harry Potter and the Philosopher's Stone",
        "author": "J.K. Rowling",
        "user:preferences": updated_preferences  # Updated!
    }
)

print(f"✅ Session 3 created with updated preferences")
print(f"\n   Changes:")
print(f"     travels_with_kids: False → True")
print(f"     preferred_pace: relaxed → moderate")
print(f"\n   These new preferences will be used in future sessions!")

✅ Session 3 created with updated preferences

   Changes:
     travels_with_kids: False → True
     preferred_pace: relaxed → moderate

   These new preferences will be used in future sessions!


## Scenario 8: Multi-User Support

Different users have isolated sessions and preferences.

In [25]:
# Create session for a different user
bob_session_id = str(uuid.uuid4())
bob_user_id = "bob"

bob_preferences = {
    "prefers_museums": False,
    "budget": "luxury",
    "favorite_genres": ["science fiction"],
    "preferred_pace": "fast-paced"
}

await session_service.create_session(
    app_name="storyland",
    user_id=bob_user_id,
    session_id=bob_session_id,
    state={
        "book_title": "Dune",
        "author": "Frank Herbert",
        "user:preferences": bob_preferences
    }
)

print(f"✅ Created session for user '{bob_user_id}'")
print(f"\n   Alice's preferences:")
print(f"     - Museums: {user_preferences['prefers_museums']}")
print(f"     - Budget: {user_preferences['budget']}")
print(f"     - Pace: {user_preferences['preferred_pace']}")

print(f"\n   Bob's preferences (completely different):")
print(f"     - Museums: {bob_preferences['prefers_museums']}")
print(f"     - Budget: {bob_preferences['budget']}")
print(f"     - Pace: {bob_preferences['preferred_pace']}")

print(f"\n   Each user's data is completely isolated!")

✅ Created session for user 'bob'

   Alice's preferences:
     - Museums: True
     - Budget: moderate
     - Pace: relaxed

   Bob's preferences (completely different):
     - Museums: False
     - Budget: luxury
     - Pace: fast-paced

   Each user's data is completely isolated!


## Scenario 9: Inspect Database

Check what's actually in the database.

In [26]:
# Get database statistics
conn = sqlite3.connect('storyland_sessions.db')
cursor = conn.cursor()

# Total sessions
cursor.execute("SELECT COUNT(*) FROM sessions")
total_sessions = cursor.fetchone()[0]

# Unique users
cursor.execute("SELECT COUNT(DISTINCT user_id) FROM sessions")
unique_users = cursor.fetchone()[0]

# Sessions per user
cursor.execute("""
    SELECT user_id, COUNT(*) as session_count
    FROM sessions
    GROUP BY user_id
    ORDER BY session_count DESC
""")
user_stats = cursor.fetchall()

conn.close()

print(f"📊 DATABASE STATISTICS")
print("=" * 60)
print(f"\n   Total sessions: {total_sessions}")
print(f"   Unique users: {unique_users}")
print(f"\n   Sessions per user:")
for user, count in user_stats:
    print(f"     - {user}: {count} sessions")

📊 DATABASE STATISTICS

   Total sessions: 11
   Unique users: 3

   Sessions per user:
     - alice: 7 sessions
     - user1: 2 sessions
     - bob: 2 sessions


## Summary

This notebook demonstrated:

✅ **Scenario 1**: Creating sessions with user preferences  
✅ **Scenario 2**: Automatic preference loading for returning users  
✅ **Scenario 3**: Querying memory for personalization  
✅ **Scenario 4**: Resuming previous sessions  
✅ **Scenario 5**: Direct SQL queries for user history  
✅ **Scenario 6**: Building comprehensive user profiles  
✅ **Scenario 7**: Updating user preferences over time  
✅ **Scenario 8**: Multi-user isolation and support  
✅ **Scenario 9**: Database inspection and statistics  

### Key Takeaways

1. **User-scoped state** (`user:preferences`) persists across sessions
2. **Session state** is specific to each conversation
3. **Memory service** allows querying past interactions
4. **Database persistence** enables analytics and user profiling
5. **Multi-user support** keeps data isolated and secure

### Next Steps

- Integrate reader profile agent for automatic personalization
- Build recommendation engine based on user history
- Create analytics dashboard for user insights
- Implement preference learning from feedback

## Cleanup (Optional)

Remove test data if needed.

In [22]:
# Uncomment to delete test sessions
# import os
# if os.path.exists('storyland_sessions.db'):
#     os.remove('storyland_sessions.db')
#     print("✅ Database deleted")

print("Run the cell above to clean up test data")

Run the cell above to clean up test data
